# LAYER WISE PROPAGATION RELEVANCE

In [1]:
import os
import torch
import copy
import shap
import random
import pickle

import pandas as pd
import numpy as np
from PIL import Image

from data.brain_tumor_mri import BrainTumorMRI
from torchvision import transforms
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader


import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from models.vgg import VGG
from models.vgg_deep import DVGG
from models.cnn import ConvNet
from models.cnn_deep import DConvNet

from helpers.prediction import get_labels_from_prediction

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
transformations = transforms.Compose([
    transforms.Resize((255, 255)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    ),
])
    
def image_to_features(img_tensor):
    img_transformed = transformations(img_tensor)
    img_batch = img_transformed.unsqueeze(0)
    return img_batch

default_cmap = LinearSegmentedColormap.from_list('custom blue', 
                                                 [(0, '#ffffff'),
                                                  (0.25, '#252b36'),
                                                  (1, '#000000')], N=256)

In [3]:
BATCH_SIZE = 64
VAL_SIZE = 0.3
EPOCHS = 60
DATASET_RANDOM = 37

brain_tumor = BrainTumorMRI(val_size=VAL_SIZE, batch_size=BATCH_SIZE, random_state=DATASET_RANDOM, transformations=transformations)

In [4]:
print(len(brain_tumor.train_dataset), len(brain_tumor.test_dataset), len(brain_tumor.train_dataset)+len(brain_tumor.test_dataset))

5712 1311 7023


In [6]:
from collections import Counter
print(Counter(brain_tumor.train_dataset.targets), Counter(brain_tumor.test_dataset.targets), Counter(brain_tumor.test_dataset.targets + brain_tumor.train_dataset.targets))

Counter({2: 1595, 3: 1457, 1: 1339, 0: 1321}) Counter({2: 405, 1: 306, 0: 300, 3: 300}) Counter({2: 2000, 3: 1757, 1: 1645, 0: 1621})


In [4]:
# dataset = brain_tumor.test_dataset
# dataloader = brain_tumor.test_dataloader
# data, targets = get_labels_from_prediction(dataloader)

# samples = pd.DataFrame(dataset.imgs, columns=["img", "target"]).groupby("target").sample(1, random_state=6).reset_index()
# test_samples = samples["img"].apply(Image.open).apply(lambda x: x.convert('RGB')).tolist()

## LOADING MODELS

In [4]:
CONV_PATH = "experiments-1/convnet/version_3/checkpoints/convnet-epoch=54-train_loss=0.2748-train_acc=0.8954-val_loss=0.4169-val_acc=0.8530.ckpt"
conv_state = torch.load(CONV_PATH)
conv = ConvNet.load_from_checkpoint(CONV_PATH)

VGG_PATH = "experiments-1/vgg/version_0/checkpoints/vgg-epoch=59-train_loss=0.3602-train_acc=0.8647-val_loss=0.3324-val_acc=0.8851.ckpt"
vgg_state = torch.load(VGG_PATH)
vgg = VGG.load_from_checkpoint(VGG_PATH)

DVGG_PATH = "./experiments-1/dvgg/version_0/checkpoints/dvgg-epoch=21-train_loss=0.1167-train_acc=0.9650-val_loss=0.0649-val_acc=0.9837.ckpt"
dvgg_state = torch.load(DVGG_PATH)
dvgg = DVGG.load_from_checkpoint(DVGG_PATH)

# with open("./experiments-1/dvgg/version_0/dvgg-timing.dump", "rb") as f:
#     dvgg_timers = pickle.load(f)

DVGG_PATH = "./experiments-1/dvgg/version_1/checkpoints/dvgg-epoch=26-train_loss=0.1616-train_acc=0.9647-val_loss=0.0605-val_acc=0.9866.ckpt"
dvgg_pca_state = torch.load(DVGG_PATH)
dvgg_pca = DVGG.load_from_checkpoint(DVGG_PATH)

with open("./experiments-1/dvgg/version_1/dvgg-timing.dump", "rb") as f:
    dvgg_pca_timers = pickle.load(f)

In [9]:
models = [conv, vgg, dvgg, dvgg_pca]
models_states = [conv_state, vgg_state, dvgg_state,  dvgg_pca_state]

## SUMMARY

In [5]:

import math

millnames = ['',' k',' M',' B',' Tr']

def millify(n):
    n = float(n)
    millidx = max(0,min(len(millnames)-1,
                        int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))

    return '{:.0f}{}'.format(n / 10**(3 * millidx), millnames[millidx])

def time_format(state):
    elapsed_time = int(state["callbacks"]["Timer"]["time_elapsed"]["train"] // (60 * 60))
    return f"{elapsed_time} h"

def estimated_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    size_all_mb = (param_size + buffer_size) / 1024**2
    return size_all_mb, f'{size_all_mb:.3f} MB'

In [10]:
names = [model.name for model in models]

data_summary = {
    "Number of classes": [model.num_classes for model in models],
    "Number of groups": [str(model.ngroups) if hasattr(model, "ngroups") else "-" for model in models],
    "Number of parameters": [millify(sum(p.numel() for p in model.parameters())) for model in models],
    "Estimated model size": [estimated_model_size(model)[1] for model in models],
    "PCA": [str(model.pca) if (hasattr(model, "pca") and hasattr(model, "preprocessing") and model.preprocessing) else "-" for model in models],
    "Training Time": [ time_format(state) for state in models_states],
}

models_summary = pd.DataFrame(data=data_summary, index=names)

In [ ]:
models_summary

In [11]:
print(models_summary.to_latex(position="h", multirow=True, caption="Parâmetros dos modelos do experimento."))

\begin{table}[h]
\caption{Parâmetros dos modelos do experimento.}
\begin{tabular}{lrlllll}
\toprule
 & Number of classes & Number of groups & Number of parameters & Estimated model size & PCA & Training Time \\
\midrule
convnet & 4 & - & 16 M & 62.037 MB & - & 2 h \\
vgg & 4 & - & 134 M & 512.226 MB & - & 17 h \\
dvgg & 4 & 4 & 134 M & 512.226 MB & - & 19 h \\
dvgg & 8 & 8 & 134 M & 512.288 MB & 64 & 21 h \\
\bottomrule
\end{tabular}
\end{table}



## PREDICTIONS

In [12]:
import joblib
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from pytorch_lightning import Trainer

In [39]:
accs = []
f1s = []

for model in models:
    trainer = Trainer()
    batch_predictions = trainer.predict(model, dataloaders=brain_tumor.test_dataloader)
    truth_labels, preds_labels = get_labels_from_prediction(batch_predictions)
    accs.append(accuracy_score(truth_labels, preds_labels))
    f1s.append(f1_score(truth_labels, preds_labels, average="macro"))

data_report = {
    "Acurácia": accs,
    "F1 Score": f1s,
}

report_summary = pd.DataFrame(data=data_report, index=names)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/arthuralvim/Work/xai-clustering/lightning_logs


Predicting: 0it [00:00, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

In [40]:
print(report_summary.to_latex(position="h", multirow=True, caption="Acurácia e F1 dos modelos do experimento."))

\begin{table}[h]
\caption{Acurácia e F1 dos modelos do experimento.}
\begin{tabular}{lrr}
\toprule
 & Acurácia & F1 Score \\
\midrule
convnet & 0.848970 & 0.842662 \\
vgg & 0.855072 & 0.845358 \\
dvgg & 0.987796 & 0.987219 \\
dvgg & 0.989321 & 0.988756 \\
\bottomrule
\end{tabular}
\end{table}



In [15]:
with open("./experiments-1/dvgg/version_1/artifacts/26/features-targets.dump", "rb") as f:
    dvgg_pca_targets = np.load(f)
dvgg_pca_clu = joblib.load("./experiments-1/dvgg/version_1/artifacts/26/clustering.dump")
adjusted_rand_score(dvgg_pca_targets, dvgg_pca_clu.labels_), adjusted_mutual_info_score(dvgg_pca_targets, dvgg_pca_clu.labels_)

(0.603025398078843, 0.752158799197294)

In [16]:
with open("./experiments-1/dvgg/version_0/artifacts/21/features-targets.dump", "rb") as f:
    dvgg_targets = np.load(f)
dvgg_clu = joblib.load("./experiments-1/dvgg/version_0/artifacts/21/clustering.dump")
adjusted_rand_score(dvgg_targets, dvgg_clu.labels_), adjusted_mutual_info_score(dvgg_targets, dvgg_clu.labels_)

(0.8546398198366183, 0.8297424480896636)